In [5]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from notebookutils import mssparkutils
from pyspark.sql.types import *
from pyspark.sql.functions import *

#create a folder
inputPath = 'Files/data/'
mssparkutils.fs.mkdirs(inputPath)

#Create a stream that reads data from the folder, using a JASON schema
jasonSchema = StructType([
    StructField("device",StringType(), False),
    StructField("status",StringType(), False)
])

iostream = spark.readStream.schema(jasonSchema).option("maxFilePertrigger",1).json(inputPath)

# Write some event data to the folder
device_data = '''{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev2","status":"error"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"error"}
{"device":"Dev2","status":"ok"}
{"device":"Dev2","status":"error"}
{"device":"Dev1","status":"ok"}'''

mssparkutils.fs.put(inputPath + "data.txt", device_data,True)
print("Source stream created ...")

StatementMeta(, fdce1c8b-b9ab-4265-bc60-fff4a96fcca7, 7, Finished, Available)

Source stream created ...


In [6]:
# Write the stream to a delta table.
delta_stream_table_path = 'Tables/ioDeviceData'
checkpointPath = 'Files/delta/checkpoint'

deltaStream = iostream.writeStream.format("delta").option("checkpointLocation", checkpointPath).start(delta_stream_table_path)

print("streaming to delta sink...")

StatementMeta(, fdce1c8b-b9ab-4265-bc60-fff4a96fcca7, 8, Finished, Available)

streaming to delta sink...


In [9]:
%%sql
SELECT * FROM ioDeviceData

StatementMeta(, fdce1c8b-b9ab-4265-bc60-fff4a96fcca7, 11, Finished, Available)

<Spark SQL result set with 16 rows and 2 fields>

In [8]:
# Add more data to the source stream
more_data = '''{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"error"}
{"device":"Dev2","status":"error"}
{"device":"Dev1","status":"ok"}'''

mssparkutils.fs.put(inputPath + "more-data.txt", more_data, True)

StatementMeta(, fdce1c8b-b9ab-4265-bc60-fff4a96fcca7, 10, Finished, Available)

True

In [11]:
deltaStream.stop()

StatementMeta(, fdce1c8b-b9ab-4265-bc60-fff4a96fcca7, 13, Finished, Available)